Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


In [ ]:
import pandas as pd
import numpy as np
import warnings
import torch
from torch import autograd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

warnings.filterwarnings('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1XNbvt_Sqf8RVuCJEi4taWU7pPjYheppt"})
downloaded.GetContentFile('item_categories.csv')
downloaded = drive.CreateFile({'id':"1ovTp6MMef5oHs8kPQ-k5tsq1QHsGfcso"})
downloaded.GetContentFile('items.csv')
downloaded = drive.CreateFile({'id':"1shLPYMyXRbDmRn_btXoNptcQkWMhS6Af"})
downloaded.GetContentFile('sales_train.csv')
downloaded = drive.CreateFile({'id':"1hgJ2oKouYHenN-SpU-dUZH7QKaJHBEop"})
downloaded.GetContentFile('shops.csv')

In [ ]:
item_categories = pd.read_csv('item_categories.csv')
items = pd.read_csv('items.csv')
sales_train = pd.read_csv('sales_train.csv')
shops = pd.read_csv('shops.csv')

Все задания по ссылке: https://docs.google.com/document/d/1IVuZfnTzgRnMjtmc8p3AEDfwYVQJK0fl-57YysOuLnk/edit

In [ ]:
# не перезапускать ячейку
from random import choice
var42 = pd.DataFrame([[choice(range(1,20))] for _ in range(20)], columns=["К34241"]).T
var40 = pd.DataFrame([[choice(range(1,20))] for _ in range(20)], columns=["К34401"]).T
display(var42.iloc[:,1:9])
display(var40.iloc[:,1:16])

,1,2,3,4,5,6,7,8
К34241,13,6,2,5,8,10,3,17


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
К34401,13,12,11,13,1,4,17,1,7,4,12,3,4,15,5


Есть ли товары, цена которых повышается каждые три месяца с начала наблюдений?

In [ ]:
# добавляем столбец - разделение на периоды по три месяца
sales_train['stage'] = sales_train['date_block_num'] // 3

In [ ]:
# оставляем items, которые продавались каждые 3 месяца
item_counted = sales_train.groupby(['item_id'])['stage'].nunique().reset_index()
items_list = list(item_counted[item_counted['stage'] == item_counted['stage'].max()]['item_id'])
sales_train_slice = sales_train[sales_train['item_id'].isin(items_list)]

In [ ]:
def issorted(s):
  l = list(s)
  return all(l[i] <= l[i+1] for i in range(len(l) - 1))

In [ ]:
res = sales_train_slice.groupby(['item_id', 'stage'])['item_price'].agg('mean').reset_index()
res = pd.DataFrame(res.groupby('item_id')['item_price'].agg(lambda l: issorted(l))).reset_index()
res[res['item_price'] == True].merge(items)

,item_id,item_price,item_name,category_id
0,226,True,1C: Audiobooks. Detectives on the road. In the...,45
1,474,True,1C: Accounting 8 CORP: accounting in the paren...,49
2,686,True,1C: Reporting Entrepreneur 8,73
3,831,True,1C: Managing our firm 8. Basic version,73
4,1128,True,ABBA Abba LP,58
5,2575,True,DEL REY LANA Born To Die The Paradise Editio...,58
6,4579,True,MCCARTNEY PAUL Kisses On The Bottom,55
7,5063,True,NIRVANA Unplugged In New York LP,58
8,5540,True,PRESLEY ELVIS Gold 2LP,58
9,6065,True,Radmin 3 - 1 persons.,75
